In [1]:
import pandas as pd 
import pyodbc
import sqlalchemy
import datetime as dt 
import numpy as np

#Se definen las conexiones

def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'jrivas',
                        'XXXXX',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res
    
def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res

In [2]:
query_1 = """ 
 
select distinct YEAR(VisitStartDateTime) as year, MONTH(VisitStartDateTime) as month, DAY(VisitStartDateTime) as day, sessionuid, sceneuid, VisitStartDateTime, locationId, SceneType, SubSceneType, TipoMueble, FabMueble, productid, Puertas_Coca, Puertas_Competencia, Puertas_Cadena from dbo.as_ir_report    
where YEAR(VisitStartDateTime) = '2021' and SceneType = 'Enfriador' and TipoMueble = 'Enfriadores Línea de Cajas' 

        """

In [3]:
df = pd.DataFrame(get_query(query_1))

In [4]:
df['date_format']=pd.to_datetime(df[['year','month','day']])

In [5]:
df=df.rename(columns={"sessionuid": "SessionUid"})

In [6]:
df.duplicated(subset='SessionUid', keep='first').sum()

1831688

In [7]:
df=df.drop_duplicates(subset=['SessionUid'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14301 entries, 0 to 554817
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   year                 14301 non-null  int64         
 1   month                14301 non-null  int64         
 2   day                  14301 non-null  int64         
 3   SessionUid           14301 non-null  object        
 4   sceneuid             14301 non-null  object        
 5   VisitStartDateTime   14301 non-null  datetime64[ns]
 6   locationId           14301 non-null  int64         
 7   SceneType            14301 non-null  object        
 8   SubSceneType         14301 non-null  object        
 9   TipoMueble           14301 non-null  object        
 10  FabMueble            14301 non-null  object        
 11  productid            14301 non-null  int64         
 12  Puertas_Coca         14301 non-null  int64         
 13  Puertas_Competencia  14301 non

In [8]:
query_2 = """ 
   
   select distinct SessionUid, LocationId, UserId, DateId, ChainDsc, Format, Clasification, ICE, coolers_fl_ko, enfriador_Competencia_FL, enfriador_Cadena_FL, is_reported from dbo.r_autoservicios_v3    

        """

In [9]:
df2 = pd.DataFrame(get_query(query_2))

In [10]:
df2.head()

,SessionUid,LocationId,UserId,DateId,ChainDsc,Format,Clasification,ICE,coolers_fl_ko,enfriador_Competencia_FL,enfriador_Cadena_FL,is_reported
0,0001df49-6f2d-4795-83fd-e961ffac4f3a,7277764,45793,20210305,SUPER AKI,MINI,BÁSICA,105.1600,1,0,0,1
1,0002f44a-be74-4b2b-954c-71ddaedde55f,6855087,45842,20210121,MERCADO,BODEGA,BÁSICA,68.8100,3,2,0,1
2,0003e1de-616b-4ccb-8af2-1f8de9957b66,6585042,45877,20210206,CASA LEY,SUPER,BÁSICA,101.6500,2,1,0,1
3,00043727-9410-49ff-8624-e571b3ad86db,6585252,45894,20201216,HIPERMERCADO,HIPER,BÁSICA,94.2500,6,2,0,1
4,0004b846-2f8d-4cbb-b927-8ba9767e4334,6860791,45929,20210825,BODEGA AURRERA,BODEGA,DESARROLLO,106.2315,1,1,0,1


In [11]:
dff = pd.merge(df2, df, on='SessionUid', how='inner')
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14301 entries, 0 to 14300
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   SessionUid                14301 non-null  object        
 1   LocationId                14301 non-null  int64         
 2   UserId                    14301 non-null  int64         
 3   DateId                    14301 non-null  object        
 4   ChainDsc                  14285 non-null  object        
 5   Format                    14301 non-null  object        
 6   Clasification             14285 non-null  object        
 7   ICE                       14301 non-null  float64       
 8   coolers_fl_ko             14301 non-null  int64         
 9   enfriador_Competencia_FL  14301 non-null  int64         
 10  enfriador_Cadena_FL       14301 non-null  int64         
 11  is_reported               14301 non-null  int64         
 12  year              

In [12]:
table = pd.pivot_table(dff, values=['coolers_fl_ko','enfriador_Competencia_FL', 'enfriador_Cadena_FL'], index=['UserId', 'LocationId', 'is_reported', 'date_format', 'month', 'day', 'SessionUid'],
                     fill_value=0)
table=pd.DataFrame(table)
table

coolers_fl_ko  \
UserId LocationId is_reported date_format month day SessionUid                                            
43976  6473384    0           2021-06-01  6     1   d2df32c6-f357-4009-ae85-5d6c38809ea7              4   
                              2021-06-24  6     24  4bd0947a-e32a-4116-9128-115508b65a53              2   
43978  6473384    0           2021-06-01  6     1   1fffc695-3619-49c5-b01b-d4510c2ec5fd              4   
                              2021-06-17  6     17  f5caea7c-a7f2-4c14-982e-8094c25eb31b              6   
43979  6473384    0           2021-06-04  6     4   d391984a-ee97-4df1-b9c0-481f941a2295              4   
...                                                                                                 ...   
58238  6853977    1           2021-10-06  10    6   2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a              3   
       6861884    1           2021-10-02  10    2   b10a9d79-30cf-4cee-b465-5107706d8409              3   
       7277788    1           2021-10-09  10    9   5b784511-8c7c-40d5-90df-8d0ad5948ed4              3   
       7277818    1           2021-10-12  10    12  3a603e1c-fce7-40ac-8244-268593dca0d0              2   
       9813407    1           2021-09-28  9     28  37aa6a75-1fcf-4c71-963e-cc64213b8f1c              2   

                                                                                          enfriador_Cadena_FL  \
UserId LocationId is_reported date_format month day SessionUid                                                  
43976  6473384    0           2021-06-01  6     1   d2df32c6-f357-4009-ae85-5d6c38809ea7                    0   
                              2021-06-24  6     24  4bd0947a-e32a-4116-9128-115508b65a53                    0   
43978  6473384    0           2021-06-01  6     1   1fffc695-3619-49c5-b01b-d4510c2ec5fd                    0   
                              2021-06-17  6     17  f5caea7c-a7f2-4c14-982e-8094c25eb31b                    0   
43979  6473384    0           2021-06-04  6     4   d391984a-ee97-4df1-b9c0-481f941a2295                    0   
...                                                                                                       ...   
58238  6853977    1           2021-10-06  10    6   2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a                    0   
       6861884    1           2021-10-02  10    2   b10a9d79-30cf-4cee-b465-5107706d8409                    0   
       7277788    1           2021-10-09  10    9   5b784511-8c7c-40d5-90df-8d0ad5948ed4                    0   
       7277818    1           2021-10-12  10    12  3a603e1c-fce7-40ac-8244-268593dca0d0                    0   
       9813407    1           2021-09-28  9     28  37aa6a75-1fcf-4c71-963e-cc64213b8f1c                    0   

                                                                                          enfriador_Competencia_FL  
UserId LocationId is_reported date_format month day SessionUid                                                      
43976  6473384    0           2021-06-01  6     1   d2df32c6-f357-4009-ae85-5d6c38809ea7                         0  
                              2021-06-24  6     24  4bd0947a-e32a-4116-9128-115508b65a53                         1  
43978  6473384    0           2021-06-01  6     1   1fffc695-3619-49c5-b01b-d4510c2ec5fd                         3  
                              2021-06-17  6     17  f5caea7c-a7f2-4c14-982e-8094c25eb31b                         3  
43979  6473384    0           2021-06-04  6     4   d391984a-ee97-4df1-b9c0-481f941a2295                         2  
...                                                                                                            ...  
58238  6853977    1           2021-10-06  10    6   2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a                         1  
       6861884    1           2021-10-02  10    2   b10a9d79-30cf-4cee-b465-5107706d8409                         3  
       7277788    1           2021-10-09  10    9   5b7845

In [13]:
table = table.reset_index()
table

,UserId,LocationId,is_reported,date_format,month,day,SessionUid,coolers_fl_ko,enfriador_Cadena_FL,enfriador_Competencia_FL
0,43976,6473384,0,2021-06-01,6,1,d2df32c6-f357-4009-ae85-5d6c38809ea7,4,0,0
1,43976,6473384,0,2021-06-24,6,24,4bd0947a-e32a-4116-9128-115508b65a53,2,0,1
2,43978,6473384,0,2021-06-01,6,1,1fffc695-3619-49c5-b01b-d4510c2ec5fd,4,0,3
3,43978,6473384,0,2021-06-17,6,17,f5caea7c-a7f2-4c14-982e-8094c25eb31b,6,0,3
4,43979,6473384,0,2021-06-04,6,4,d391984a-ee97-4df1-b9c0-481f941a2295,4,0,2
...,...,...,...,...,...,...,...,...,...,...
14296,58238,6853977,1,2021-10-06,10,6,2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a,3,0,1
14297,58238,6861884,1,2021-10-02,10,2,b10a9d79-30cf-4cee-b465-5107706d8409,3,0,3
14298,58238,7277788,1,2021-10-09,10,9,5b784511-8c7c-40d5-90df-8d0ad5948ed4,3,0,2
14299,58238,7277818,1,2021-10-12,10,12,3a603e1c-fce7-40ac-8244-268593dca0d0,2,0,2


In [14]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14301 entries, 0 to 14300
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   UserId                    14301 non-null  int64         
 1   LocationId                14301 non-null  int64         
 2   is_reported               14301 non-null  int64         
 3   date_format               14301 non-null  datetime64[ns]
 4   month                     14301 non-null  int64         
 5   day                       14301 non-null  int64         
 6   SessionUid                14301 non-null  object        
 7   coolers_fl_ko             14301 non-null  int64         
 8   enfriador_Cadena_FL       14301 non-null  int64         
 9   enfriador_Competencia_FL  14301 non-null  int64         
dtypes: datetime64[ns](1), int64(8), object(1)
memory usage: 1.1+ MB


In [15]:
moni=table.groupby(['UserId','LocationId', 'SessionUid', 'is_reported', 'date_format', 'month', 'day'])['coolers_fl_ko', 'enfriador_Cadena_FL', 'enfriador_Competencia_FL'].sum()

moni  

<ipython-input-15-53453be448a9>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  moni=table.groupby(['UserId','LocationId', 'SessionUid', 'is_reported', 'date_format', 'month', 'day'])['coolers_fl_ko', 'enfriador_Cadena_FL', 'enfriador_Competencia_FL'].sum()


coolers_fl_ko  \
UserId LocationId SessionUid                           is_reported date_format month day                  
43976  6473384    4bd0947a-e32a-4116-9128-115508b65a53 0           2021-06-24  6     24               2   
                  d2df32c6-f357-4009-ae85-5d6c38809ea7 0           2021-06-01  6     1                4   
43978  6473384    1fffc695-3619-49c5-b01b-d4510c2ec5fd 0           2021-06-01  6     1                4   
                  f5caea7c-a7f2-4c14-982e-8094c25eb31b 0           2021-06-17  6     17               6   
43979  6473384    70448b47-3f90-41a7-9f43-53b98b1aecbe 0           2021-06-23  6     23               4   
...                                                                                                 ...   
58238  6853977    2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a 1           2021-10-06  10    6                3   
       6861884    b10a9d79-30cf-4cee-b465-5107706d8409 1           2021-10-02  10    2                3   
       7277788    5b784511-8c7c-40d5-90df-8d0ad5948ed4 1           2021-10-09  10    9                3   
       7277818    3a603e1c-fce7-40ac-8244-268593dca0d0 1           2021-10-12  10    12               2   
       9813407    37aa6a75-1fcf-4c71-963e-cc64213b8f1c 1           2021-09-28  9     28               2   

                                                                                          enfriador_Cadena_FL  \
UserId LocationId SessionUid                           is_reported date_format month day                        
43976  6473384    4bd0947a-e32a-4116-9128-115508b65a53 0           2021-06-24  6     24                     0   
                  d2df32c6-f357-4009-ae85-5d6c38809ea7 0           2021-06-01  6     1                      0   
43978  6473384    1fffc695-3619-49c5-b01b-d4510c2ec5fd 0           2021-06-01  6     1                      0   
                  f5caea7c-a7f2-4c14-982e-8094c25eb31b 0           2021-06-17  6     17                     0   
43979  6473384    70448b47-3f90-41a7-9f43-53b98b1aecbe 0           2021-06-23  6     23                     0   
...                                                                                                       ...   
58238  6853977    2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a 1           2021-10-06  10    6                      0   
       6861884    b10a9d79-30cf-4cee-b465-5107706d8409 1           2021-10-02  10    2                      0   
       7277788    5b784511-8c7c-40d5-90df-8d0ad5948ed4 1           2021-10-09  10    9                      0   
       7277818    3a603e1c-fce7-40ac-8244-268593dca0d0 1           2021-10-12  10    12                     0   
       9813407    37aa6a75-1fcf-4c71-963e-cc64213b8f1c 1           2021-09-28  9     28                     0   

                                                                                          enfriador_Competencia_FL  
UserId LocationId SessionUid                           is_reported date_format month day                            
43976  6473384    4bd0947a-e32a-4116-9128-115508b65a53 0           2021-06-24  6     24                          1  
                  d2df32c6-f357-4009-ae85-5d6c38809ea7 0           2021-06-01  6     1                           0  
43978  6473384    1fffc695-3619-49c5-b01b-d4510c2ec5fd 0           2021-06-01  6     1                           3  
                  f5caea7c-a7f2-4c14-982e-8094c25eb31b 0           2021-06-17  6     17                          3  
43979  6473384    70448b47-3f90-41a7-9f43-53b98b1aecbe 0           2021-06-23  6     23                          0  
...                                                                                                            ...  
58238  6853977    2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a 1           2021-10-06  10    6                           1  
       6861884    b10a9d79-30cf-4cee-b465-5107706d8409 1           2021-10-02  10    2                           3  
       7277788    5b784511-8c7c-40d5-90df-8d0ad5948ed4 1  

In [16]:
table2 = moni.reset_index()
table2

,UserId,LocationId,SessionUid,is_reported,date_format,month,day,coolers_fl_ko,enfriador_Cadena_FL,enfriador_Competencia_FL
0,43976,6473384,4bd0947a-e32a-4116-9128-115508b65a53,0,2021-06-24,6,24,2,0,1
1,43976,6473384,d2df32c6-f357-4009-ae85-5d6c38809ea7,0,2021-06-01,6,1,4,0,0
2,43978,6473384,1fffc695-3619-49c5-b01b-d4510c2ec5fd,0,2021-06-01,6,1,4,0,3
3,43978,6473384,f5caea7c-a7f2-4c14-982e-8094c25eb31b,0,2021-06-17,6,17,6,0,3
4,43979,6473384,70448b47-3f90-41a7-9f43-53b98b1aecbe,0,2021-06-23,6,23,4,0,0
...,...,...,...,...,...,...,...,...,...,...
14296,58238,6853977,2d56fb38-f62c-42ba-9ff1-ef2f1a1f082a,1,2021-10-06,10,6,3,0,1
14297,58238,6861884,b10a9d79-30cf-4cee-b465-5107706d8409,1,2021-10-02,10,2,3,0,3
14298,58238,7277788,5b784511-8c7c-40d5-90df-8d0ad5948ed4,1,2021-10-09,10,9,3,0,2
14299,58238,7277818,3a603e1c-fce7-40ac-8244-268593dca0d0,1,2021-10-12,10,12,2,0,2


In [17]:
table2=table2.sort_values(['LocationId', 'date_format'])
table2

,UserId,LocationId,SessionUid,is_reported,date_format,month,day,coolers_fl_ko,enfriador_Cadena_FL,enfriador_Competencia_FL
11744,52049,6188351,6fbb054a-4bb4-4660-b055-9be64eb62409,1,2021-05-21,5,21,5,0,5
3895,45847,6188351,7dbcf5ac-dfb7-4a2c-9c74-62ee7957352e,1,2021-06-03,6,3,4,1,5
3894,45847,6188351,549e6699-f573-4a84-ba02-53f02693fe43,1,2021-07-06,7,6,4,0,6
11745,52049,6188351,a38e7b33-6b0f-42e3-a8bc-7991cb5fb7e0,1,2021-08-14,8,14,4,0,6
11742,52049,6188351,2c000823-ce4b-4d38-a733-37a9e06b29c7,1,2021-09-09,9,9,4,0,6
...,...,...,...,...,...,...,...,...,...,...
587,45789,10068942,4db7fd6d-fe70-42a3-a668-f15c329c2462,1,2021-09-03,9,3,1,0,0
589,45789,10068942,fbb86cc5-c34f-4a23-b47e-fe297cf2c751,1,2021-10-13,10,13,1,0,0
590,45789,10186808,139944a1-cdb9-40af-aa85-2d38155ee65b,0,2021-10-14,10,14,1,0,0
633,45790,10186809,0405514b-9307-4c87-8b4c-7872eb45ee73,1,2021-10-08,10,8,1,0,1


In [18]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14301 entries, 11744 to 591
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   UserId                    14301 non-null  int64         
 1   LocationId                14301 non-null  int64         
 2   SessionUid                14301 non-null  object        
 3   is_reported               14301 non-null  int64         
 4   date_format               14301 non-null  datetime64[ns]
 5   month                     14301 non-null  int64         
 6   day                       14301 non-null  int64         
 7   coolers_fl_ko             14301 non-null  int64         
 8   enfriador_Cadena_FL       14301 non-null  int64         
 9   enfriador_Competencia_FL  14301 non-null  int64         
dtypes: datetime64[ns](1), int64(8), object(1)
memory usage: 1.2+ MB


In [19]:
table2["Coca-Cola"] = table2["coolers_fl_ko"].shift(1)
table2["Cadena"] = table2["enfriador_Cadena_FL"].shift(1)
table2["Competencia"] = table2["enfriador_Competencia_FL"].shift(1)
table2.dtypes

UserId                               int64
LocationId                           int64
SessionUid                          object
is_reported                          int64
date_format                 datetime64[ns]
month                                int64
day                                  int64
coolers_fl_ko                        int64
enfriador_Cadena_FL                  int64
enfriador_Competencia_FL             int64
Coca-Cola                          float64
Cadena                             float64
Competencia                        float64
dtype: object

In [20]:
table2 = table2.fillna(0)

In [21]:
table2 = table2.astype({"Coca-Cola": int, "Competencia": int, "Cadena": int, "LocationId": object})

In [22]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14301 entries, 11744 to 591
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   UserId                    14301 non-null  int64         
 1   LocationId                14301 non-null  object        
 2   SessionUid                14301 non-null  object        
 3   is_reported               14301 non-null  int64         
 4   date_format               14301 non-null  datetime64[ns]
 5   month                     14301 non-null  int64         
 6   day                       14301 non-null  int64         
 7   coolers_fl_ko             14301 non-null  int64         
 8   enfriador_Cadena_FL       14301 non-null  int64         
 9   enfriador_Competencia_FL  14301 non-null  int64         
 10  Coca-Cola                 14301 non-null  int32         
 11  Cadena                    14301 non-null  int32         
 12  Competencia     

In [23]:
table2["LocationIdChanged"] = table2["LocationId"].shift(1, fill_value=table2["LocationId"].head(1)) != table2["LocationId"]
table2.dtypes

UserId                               int64
LocationId                          object
SessionUid                          object
is_reported                          int64
date_format                 datetime64[ns]
month                                int64
day                                  int64
coolers_fl_ko                        int64
enfriador_Cadena_FL                  int64
enfriador_Competencia_FL             int64
Coca-Cola                            int32
Cadena                               int32
Competencia                          int32
LocationIdChanged                     bool
dtype: object

In [24]:
table2["LocationIdChanged"] = table2["LocationIdChanged"].astype(object)
table2.dtypes

UserId                               int64
LocationId                          object
SessionUid                          object
is_reported                          int64
date_format                 datetime64[ns]
month                                int64
day                                  int64
coolers_fl_ko                        int64
enfriador_Cadena_FL                  int64
enfriador_Competencia_FL             int64
Coca-Cola                            int32
Cadena                               int32
Competencia                          int32
LocationIdChanged                   object
dtype: object

In [25]:
for OutletCodeChanged in table2:
      s = table2['LocationIdChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_CC = 'change'
              
          else: 
              Flag_CC = table2["coolers_fl_ko"] - table2["Coca-Cola"]

              table2['Flag_CC']=Flag_CC

In [26]:
for OutletCodeChanged in table2:
      s = table2['LocationIdChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_Comp = 'change'
              
          else: 
              Flag_Comp = table2["enfriador_Competencia_FL"] - table2["Competencia"]

              table2['Flag_Comp']=Flag_Comp

In [27]:
for OutletCodeChanged in table2:
      s = table2['LocationIdChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_Prop = 'change'
              
          else: 
              Flag_Prop = table2["enfriador_Cadena_FL"] - table2["Cadena"]

              table2['Flag_Cad']=Flag_Prop

In [28]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14301 entries, 11744 to 591
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   UserId                    14301 non-null  int64         
 1   LocationId                14301 non-null  object        
 2   SessionUid                14301 non-null  object        
 3   is_reported               14301 non-null  int64         
 4   date_format               14301 non-null  datetime64[ns]
 5   month                     14301 non-null  int64         
 6   day                       14301 non-null  int64         
 7   coolers_fl_ko             14301 non-null  int64         
 8   enfriador_Cadena_FL       14301 non-null  int64         
 9   enfriador_Competencia_FL  14301 non-null  int64         
 10  Coca-Cola                 14301 non-null  int32         
 11  Cadena                    14301 non-null  int32         
 12  Competencia     

In [29]:
table_final = table2[['UserId','LocationId','SessionUid', 'is_reported', 'date_format', 'month', 'day', 'LocationIdChanged', 'coolers_fl_ko', 'Flag_CC', 'enfriador_Competencia_FL', 'Flag_Comp', 'enfriador_Cadena_FL', 'Flag_Cad']]
table_final

,UserId,LocationId,SessionUid,is_reported,date_format,month,day,LocationIdChanged,coolers_fl_ko,Flag_CC,enfriador_Competencia_FL,Flag_Comp,enfriador_Cadena_FL,Flag_Cad
11744,52049,6188351,6fbb054a-4bb4-4660-b055-9be64eb62409,1,2021-05-21,5,21,False,5,5,5,5,0,0
3895,45847,6188351,7dbcf5ac-dfb7-4a2c-9c74-62ee7957352e,1,2021-06-03,6,3,False,4,-1,5,0,1,1
3894,45847,6188351,549e6699-f573-4a84-ba02-53f02693fe43,1,2021-07-06,7,6,False,4,0,6,1,0,-1
11745,52049,6188351,a38e7b33-6b0f-42e3-a8bc-7991cb5fb7e0,1,2021-08-14,8,14,False,4,0,6,0,0,0
11742,52049,6188351,2c000823-ce4b-4d38-a733-37a9e06b29c7,1,2021-09-09,9,9,False,4,0,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,45789,10068942,4db7fd6d-fe70-42a3-a668-f15c329c2462,1,2021-09-03,9,3,False,1,0,0,0,0,0
589,45789,10068942,fbb86cc5-c34f-4a23-b47e-fe297cf2c751,1,2021-10-13,10,13,False,1,0,0,0,0,0
590,45789,10186808,139944a1-cdb9-40af-aa85-2d38155ee65b,0,2021-10-14,10,14,True,1,0,0,0,0,0
633,45790,10186809,0405514b-9307-4c87-8b4c-7872eb45ee73,1,2021-10-08,10,8,True,1,0,1,1,0,0


In [30]:
table_final = table_final.fillna(0)

In [31]:
print("flags-as - Borrando los datos de la tabla")
query_clear_FL_as = """ 
    
DELETE FROM reportes.FL_tot_as

        """

flags-as - Borrando los datos de la tabla


In [32]:
def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'jrivas',
                        '8khHAM=B=q$ZGr6N',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res
    
def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res

eng = get_engine()
conn = eng.connect()
conn.execute(query_clear_FL_as)
conn.close()
#get_query(query_clear_Exh_as_cc_comp)


print("flags-as - Llenando la tabla")

flags-as - Llenando la tabla


In [33]:
engine = get_engine()
table_final.to_sql('FL_tot_as', con=engine, schema='reportes', if_exists='append', index=False)